In [1]:
import jax
import jax.numpy as jnp
from tqdm import tqdm
import netket as nk
import netket.jax as nkjax
# import netket_pro as nkp
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from tqdm import tqdm
import jax.numpy as jnp
import jax
from grad_sample.utils.misc import compute_eloc
from grad_sample.utils.distances import curved_dist, fs_dist, param_overlap
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.core.global_hydra import GlobalHydra
from hydra.utils import instantiate
from grad_sample.tasks.fullsum_analysis import FullSumPruning
from grad_sample.tasks.fullsum_train import Trainer
from grad_sample.utils.is_distrib import *
from grad_sample.utils.plotting_setup import *
from grad_sample.is_hpsi.expect import *
from grad_sample.is_hpsi.qgt import QGTJacobianDenseImportanceSampling
from grad_sample.is_hpsi.operator import IS_Operator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
if GlobalHydra().is_initialized():
    GlobalHydra().clear()
with initialize(version_base=None, config_path="config_xxz/.hydra/"):
    cfg = compose(config_name="config")
    OmegaConf.set_struct(cfg, True)
    print(cfg)
    print(cfg.task)
    # cfg = OmegaConf.to_yaml(cfg)
    # take any task from cfg and run it
# analysis = FullSumPruning(cfg)
trainer = Trainer(cfg)

{'device': '2', 'solver_fn': {'_target_': 'netket.optimizer.solver.cholesky'}, 'lr': 0.005, 'n_iter': 2000, 'chunk_size_vmap': 100, 'save_every': 10, 'sample_size': 16, 'base_path': '/scratch/.amisery/grad_sample/', 'model': {'_target_': 'grad_sample.models.heisenberg.Heisenberg1d', 'J': 1.0, 'L': 14, 'sign_rule': False, 'acting_on_subspace': 0}, 'diag_shift': 0.0001, 'chunk_size_jac': 572, 'ansatz': {'_target_': 'netket.models.RBM', 'alpha': 3, 'param_dtype': 'complex'}, 'task': {'_target_': 'grad_sample.tasks.fullsum_train.Trainer'}}
{'_target_': 'grad_sample.tasks.fullsum_train.Trainer'}
{'_target_': 'netket.models.RBM', 'alpha': 3, 'param_dtype': 'complex'}
MC state loaded, num samples 10304
The ground state energy is: -25.054198134188105


In [4]:
trainer.vstate.expect(trainer.model.H)

13.9910-0.0023j ± 0.0023 [σ²=0.0550]

In [5]:
H_sq = trainer.model.H @ trainer.model.H

In [7]:
trainer.vstate.expect(H_sq)

195.793-0.055j ± 0.051 [σ²=26.124]

In [11]:
gamma = trainer.vstate.samples
if gamma.ndim >= 3:
    # use jit so that we can do it on global shared array
    gamma = jax.jit(jax.lax.collapse, static_argnums=(1, 2))(gamma, 0, 2)
apply_fun = trainer.vstate.model.apply
params = {"params" : trainer.vstate.parameters}
log_psi_gamma = apply_fun(params, gamma)
sig_conn, H_sig_gamma = trainer.model.H.get_conn_padded(gamma)
log_psi_sigma = apply_fun(params, sig_conn)

eta_conn , H_sig_eta = trainer.model.H.get_conn_padded(sig_conn)
print(H_sig_eta.shape)
log_psi_eta = apply_fun(params, eta_conn)

q_sig_gamma = H_sig_gamma.conj()/jnp.expand_dims(jnp.exp(log_psi_gamma), axis=-1)*jnp.sum(H_sig_eta * jnp.exp(log_psi_eta), axis= -1)

A_sig = jnp.sum(H_sig_eta * jnp.exp(log_psi_eta- jnp.expand_dims(log_psi_sigma, axis=-1)), axis=-1)
print(A_sig.shape)
# Mc estim of energy
H_mean = jnp.sum(q_sig_gamma*(1/A_sig.conj()))/gamma.size

# H_sq estim:
sq_conn , H_sq_mels = H_sq.get_conn_padded(gamma)
log_psi_sq_conn = apply_fun(params, sq_conn)
H_sq_loc = jnp.sum(H_sq_mels * jnp.exp(log_psi_sq_conn - jnp.expand_dims(log_psi_gamma, axis=-1)), axis=-1)
H_sq_mean = jnp.mean(H_sq_loc)
print(H_sq_mean)
print(H_mean/H_sq_mean)

(10304, 15, 15)
(10304, 15)
(195.79269124961968-0.055027856627976726j)
(0.005104167452068567-5.896381272834455e-07j)
